In [ ]:
import Pkg
#Pkg.add("MathOptInterface")
#Pkg.add("GLPK")
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using JuMP
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

In [ ]:
include("../src/read_data.jl")
include("../JuMP/main_JuMP.jl")

In [ ]:
T=P+1
PP=[p for p in 1:P]
PPP=[p for p in 0:T]
PP0=[p for p in 0:P]
PPT=[p for p in 1:T]
QQ=[q for q in 1:Q]
CC=[c for c in 1:C]


function travel_time(current_bay::Int, target_bay::Int, CTS::Constants)
    if current_bay == 0 || target_bay == 0
        return(0)
    else
        return(abs(current_bay - target_bay)*CTS.tt)
    end
end

#create bj_dict
bj_dict = Dict{Int, Int}()
bj_init = Dict{Int, Int}()
for p in PP
    bj_dict[p] = bj[p]
end

bj_dict[0] = 0
bj_dict[T] = 0
bj_init[1] = 1
bj_init[2] = 4
bj_dict

In [ ]:
model = Model(optimizer_with_attributes(GLPK.Optimizer));
# Old syntax: model = Model(solver=GLPKSolverLP(msg_lev = 4)))

In [ ]:
#if positions p, s are performed consecutively by crane q
@variable(model, x[p=0:T, s=0:T, q=1:Q], Bin)
#if positions p is filled with container i
@variable(model, w[p=0:T, i=1:C], Bin)
#if position s starts after the completion time of task p
@variable(model, z[p=1:P, s=1:P], Bin)

#completion time of task p
@variable(model, t_load[p=0:T], lower_bound=0, upper_bound=CTS.H)
#completion required time to perform task p
@variable(model, t_task[p=0:T], lower_bound=0, upper_bound=CTS.H)
#completion time of crane q
#@variable(model, t_crane[q=1:Q], lower_bound=0, upper_bound=CTS.H)
#makespan
#@variable(model, makespan, lower_bound=0, upper_bound=CTS.H)


#objective function
#@objective(model, Min, 0.8*t_load[T] + 0.2*sum(t_crane[q] for q in QQ))
@objective(model, Min, t_load[T])

In [ ]:
@constraint(model, [p=0:T, q=1:Q], x[p,p,q] == 0)
@constraint(model, [p=1:P], z[p,p] == 0)

@constraint(model, t_load[0] == 0)
@constraint(model, t_load[T] <= CTS.H)

In [ ]:
@constraint(model, t_task[0] == 0)
@constraint(model, t_task[T] == 0)

@constraint(model, [p=1:P], t_task[p] == sum(task_times[p,i]*w[p,i] for i in subset_pos(CC, tasks_by_position, p)))

In [ ]:
@constraint(model, [i=1:C], w[0,i] == 0)
@constraint(model, [i=1:C], w[T,i] == 0)

@constraint(model, [i=1:C], sum(w[p,i] for p in subset_pos(PP, tasks_by_position, i)) == 1)
@constraint(model, [p=1:P], sum(w[p,i] for i in subset_pos(CC, tasks_by_position, p)) == 1)

In [ ]:
#@constraint(model, [q=1:Q], t_crane[q] <= t_load[T])

In [ ]:
@constraint(model, [q=1:Q], sum(x[0,s,q] for s in PPT) == 1)
@constraint(model, [q=1:Q], sum(x[p,T,q] for p in PP0) == 1)

In [ ]:
@constraint(model, [p=1:P], sum(sum(x[p,s,q] for s in PPT) for q in QQ) == 1)

@constraint(model, [p=1:P, q=1:Q], sum(x[s,p,q] for s in PP0) - sum(x[p,s,q] for s in PPT) == 0)

In [ ]:
for s in PP
    for p in prec[s]
        @constraint(model, t_load[p] + 2*t_task[s] - t_load[s] <= 0)
        @constraint(model, z[p,s] + z[s,p] == 1)
    end
end

In [ ]:
for p in PPP
    for s in PPP
        if p != s
            for q in QQ
                @constraint(model, t_load[p] + travel_time(bj_dict[p], bj_dict[s], CTS) + 2*t_task[s] - t_load[s] <= CTS.H*(1-x[p,s,q]))
            end
        end
    end
end

for p in PP
    for s in PP
        if p != s
            @constraint(model, t_load[p] + 2*t_task[s] - t_load[s] <= CTS.H*(1-z[p,s]))
            @constraint(model, t_load[s] - 2*t_task[s] - t_load[p] <= CTS.H*z[p,s])

            if bj_dict[p] == bj_dict[s]
                custom_set=Array{Int,1}()
                custom_set = collect(setdiff(Set(PP0),Set(bay_to_pos(bj_dict[p], bj))))
                @constraint(model, t_load[p] + 2*t_task[s] - t_load[s] + CTS.tt*sum(sum(x[u,s,q] for u in custom_set) for q in QQ) <= CTS.H*(1-z[p,s]))
                @constraint(model, t_load[s] - 2*t_task[s] - t_load[p] + CTS.tt*sum(sum(x[u,s,q] for u in custom_set) for q in QQ) <= CTS.H*z[p,s])
            end
        end
    end
end

In [ ]:
#@constraint(model, [p=1:P, q=1:Q], t_load[p] + travel_time(p, bj_start_end[1, q], CTS) - t_crane[q] <= CTS.H*(1-x[p,T,q]))
#@constraint(model, [p=1:P, q=1:Q], t_load[p] - t_crane[q] <= CTS.H*(1-x[p,T,q]))

@constraint(model, [s=1:P, q=1:Q], travel_time(bj_init[q], s, CTS) + 2*t_task[s] - t_load[s] <= CTS.H*(1-x[0,s,q]))
#@constraint(model, [s=1:P, q=1:Q], t_task[s] - t_load[s] <= CTS.H*(1-x[0,s,q]))

In [ ]:
for p in PP
    for s in PP
        if p != s && bj_dict[p] < bj_dict[s]
            for q in QQ
                @constraint(model, sum(sum(x[u,s,v] for u in PP0) for v in collect(1:q)) - sum(sum(x[u,p,v] for u in PPT) for v in collect(1:q)) <= CTS.H*(z[p,s] + z[s,p]))
            end
        end
    end
end

## Solution

In [ ]:
JuMP.optimize!(model) # Old syntax: status = JuMP.solve(model)

In [ ]:
makespan = JuMP.value.(t_load)[T]

In [ ]:
sol_x = Dict{Int, Array}()
for q=1:Q
    for p=0:T
        for s=0:T
            if JuMP.value.(x)[p,s,q] == 1
                if haskey(sol_x, q) == false
                    sol_x[q] = Array{NamedTuple{(:load_time, :pos, :next_pos, :qc),Tuple{Int64,Int64,Int64,Int64}}, 1}()
                end
                push!(sol_x[q], (load_time=JuMP.value.(t_load)[p], pos=p, next_pos=s, qc=q))
            end                
        end
    end
    sort!(sol_x[q], by = x->x.load_time)
end

sol_x[1]

In [ ]:
sol_x[2]

In [ ]:
sol_w = Dict{Int, Int}()
for p=1:P
    for i=1:C
        if JuMP.value.(w)[p,i] == 1
            sol_w[p] = i             
        end
    end
end
sol_w[0] = 0
sol_w[T] = 0

sol_w

In [ ]:
LS = init_ls(CTS)

In [ ]:
function update_ls(LS::LoadingSequence, t::NamedTuple{(:load_time, :pos, :next_pos, :qc),Tuple{Int64,Int64,Int64,Int64,}}, sol_w::Dict{Int, Int}, CTS::Constants)
    start_time = t.load_time-2*JuMP.value.(t_task)[t.pos]
    if t.pos != 0
        LS.len += 1
        LS.tasks_left -= 1
        push!(LS.filled_pos, t.pos)
        push!(LS.loaded_cont, sol_w[t.pos])
    end
    push!(LS.order, (Task(t.pos, bj_dict[t.pos], sol_w[t.pos], JuMP.value.(t_task)[t.pos]), start_time, t.qc))
end

In [ ]:
while LS.tasks_left > 0
    task = [(load_time=CTS.H+1,pos=0,next_pos=0,qc=0)]
    for q=1:Q
        if sol_x[q][1].load_time < task[1].load_time
            task = [sol_x[q][1]]
        elseif sol_x[q][1].load_time == task[1].load_time
            push!(task, sol_x[q][1])
        end
    end

    for t in task
        update_ls(LS, t, sol_w, CTS)
        if sol_x[t.qc][1] == t
            deleteat!(sol_x[t.qc], 1)
        end
    end
end

In [ ]:
LS

In [ ]:
for p in PP
    print(JuMP.value.(t_task)[p])
    print("\n")
end

In [ ]:
function check_tasks(LS::LoadingSequence, CTS::Constants)
    if length(LS.filled_pos) == P && length(LS.loaded_cont) == C && length(setdiff(LS.filled_pos, collect(1:P))) == 0 && length(setdiff(LS.loaded_cont, collect(1:C))) == 0
        return(true)
    else
        return(false)
    end
end

function check_clearance(QC::Array{QuayCrane, 1}, q::Int, target_bay::Int, CTS::Constants)
    if QC[q-1].current_bay < target_bay - CTS.delta && QC[q+1].current_bay > target_bay + CTS.delta
        return(true)
    end
    return(false)
end

function check_prec(LS::LoadingSequence, task::Task, precedences::Dict)
    for req in precedences[task.p]
        if !(req in LS.filled_pos)
            print("trying to add position, but still missing")
            print(setdiff(precedences[task.p],LS.filled_pos))
            return(false)
        end
    end
    return(true)
end


In [ ]:
check_tasks(LS,CTS)

In [ ]:
for task in LS.order
    if task[1].p != 0
        if check_prec(LS,task[1],prec) == false
            return(false)
        end
    end
end
return(true)